'created' then did 'anything' - Android

In [1]:
%matplotlib inline

In [1]:
COHORT_QUERY = "SELECT C4 FROM %s WHERE C5 = 'account.created' AND C3 = 'Android'"

In [1]:
REST_QUERY = "SELECT C4 FROM %s"

In [1]:
TITLE = "'created' then did 'anything' - Android"

In [1]:
EVENT_STORAGE = "s3n://net-mozaws-prod-us-west-2-pipeline-analysis/fxa-retention/data/"

In [1]:
import os
from datetime import date, timedelta, datetime
import pandas

IN_IPYTHON = True
try:
    __IPYTHON__
except NameError:
    IN_IPYTHON = False

today = datetime.strptime('2015-09-28', '%Y-%m-%d').date()
last_monday = today - timedelta(days=-today.weekday(), weeks=1)

WEEK_RANGE = pandas.date_range(end=last_monday, periods=15, freq='W-MON')
# TODO for now: from events-2015-06-15.csv to events-2015-09-21.csv
WEEKS = WEEK_RANGE.map(lambda x: x.strftime('%Y-%m-%d'))

if not IN_IPYTHON:
    EVENT_STORAGE = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..', '..', 'tools', 'out')

OUT_DATA = []
VOLUME_DATA = []

for v in range(0, len(WEEKS)):
    VOLUME_DATA.append([0] * len(WEEKS))

for x in range(0, len(WEEKS)):
    OUT_DATA.append([0] * len(WEEKS))

def week_file(storage, week):
    return os.path.join(storage, 'events-' + week + '.csv')


In [1]:
import pandas, os, sys
# Only initialize Spark if testing locally
# Otherwise it should be already running within Spark
try:
    from pyspark import SparkContext
except ImportError:
    import dev_env

from pyspark import SparkContext
from pyspark.sql import SQLContext, Row

IN_IPYTHON = True

try:
    __IPYTHON__
except NameError:
    IN_IPYTHON = False
    sc = SparkContext('local')
    sys.path.append(os.path.realpath(os.curdir))
    from ipynb_generators.parts.weeks import WEEKS, WEEK_RANGE, OUT_DATA, VOLUME_DATA, EVENT_STORAGE, week_file
    COHORT_QUERY = "SELECT C4 FROM %s WHERE C5 = 'account.created'"
    REST_QUERY = "SELECT C4 FROM %s"
    print "Not in IPython, creating SparkContext manually, using fake data!"

# sc will be global in IPython
sqlContext = SQLContext(sc)

for x in range(0, len(WEEKS)):
    saved_uids = None
    saved_uids_count = None

    idx = 0
    for week in WEEKS[x:]:
        df = sqlContext.load(source='com.databricks.spark.csv', header='false', path=week_file(EVENT_STORAGE, week))
        table_name = 'week' + week.replace('-', '_')
        df.registerTempTable(table_name)

        if not saved_uids:
            signed_events = sqlContext.sql(COHORT_QUERY % table_name)
            new_uids = signed_events.map(lambda p: p.C4).distinct()

            saved_uids = new_uids
            saved_uids_count = int(new_uids.count())
            VOLUME_DATA[x][idx] = saved_uids_count
            OUT_DATA[x][idx] = 100
        else:
            created_events = sqlContext.sql(REST_QUERY % table_name)
            new_uids_created_events = created_events.map(lambda p: p.C4).distinct()

            retention_uids = saved_uids.intersection(new_uids_created_events)
            if saved_uids_count > 0:
                percentage = int((float(retention_uids.count()) / float(saved_uids_count)) * 100)
            else:
                percentage = 0
            OUT_DATA[x][idx] = percentage
            VOLUME_DATA[x][idx] = int(retention_uids.count())
        idx += 1

DATA_FRAME = pandas.DataFrame(OUT_DATA, index=WEEK_RANGE, columns=range(0, len(WEEKS)))
VOLUME_DATA_FRAME = pandas.DataFrame(VOLUME_DATA, index=WEEK_RANGE, columns=range(0, len(WEEKS)))

if not IN_IPYTHON:
    print DATA_FRAME
    print VOLUME_DATA_FRAME


In [1]:
import seaborn
import matplotlib.pyplot as plt

IN_IPYTHON = True
try:
    __IPYTHON__
except NameError:
    IN_IPYTHON = False

if IN_IPYTHON:
    seaborn.set(style='white')
    plt.figure(figsize=(16, 12))
    plt.title(TITLE, { 'fontsize': 26 })
    seaborn.heatmap(DATA_FRAME, annot=True, fmt='d', yticklabels=WEEKS, xticklabels=range(0, len(WEEKS)))
    # Rotate labels
    locs, labels = plt.yticks()
    plt.setp(labels, rotation=0)
    # Set axis font
    font = {
        'weight': 'bold',
        'size': 22
    }
    # Label axis
    plt.ylabel('Cohort Starting Week', **font)
    plt.xlabel('Retention Weeks', **font)

    print VOLUME_DATA_FRAME
